In [61]:
import pandas as pd
import numpy as np
import os

In [62]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/SALT/Dashboards/')

<h1 style="color:blue;">
    Raw Scores
</h1>

In [63]:
dfp = pd.read_csv('./CBAScoredFiles/g8.csv')

In [64]:
#check
dfp.shape

(411633, 190)

In [65]:
#df.to_pickle('./CBAScoredFiles/cba_data_for_dashboard.pkl')

In [66]:
#dfp = pd.read_pickle('./cba_data_for_dashboard.pkl')

In [67]:
unwanted = ('student_id','disecode','student_name','class','section','roll_no','unique_id','gender','social_category','medium_id'
            ,'Language_paperset','Language_jumble_set','Language_present_absent','LanguageTotal','LanguagePercentage_old','LanguagePercentage','Language_cheating_drop'
            ,'Maths_paperset','Maths_jumble_set','Maths_present_absent','MathsTotal','MathsPercentage_old','MathsPercentage','Maths_cheating_drop'
            ,'EVS_paperset','EVS_jumble_set','EVS_present_absent','EVSTotal','EVSPercentage_old','EVSPercentage','EVS_cheating_drop'
            ,'Science_paperset','Science_jumble_set','Science_present_absent','ScienceTotal','SciencePercentage_old','SciencePercentage','Science_cheating_drop'
            ,'SS_paperset','SS_jumble_set','SS_present_absent','SSTotal','SSPercentage_old','SSPercentage','SS_cheating_drop'
           )

In [68]:
var_list = list(dfp)

In [69]:
#EQ = EVS, MQ = Maths, LQ = Language (Telugu), SQ = Science, OQ = Social Science
prefixes = ('Score_EQ','Score_MQ','Score_LQ','Score_SQ','Score_OQ')
dropped = [x for x in var_list if x.startswith(prefixes)]

In [70]:
dfp.drop(dropped, axis = 1 , inplace = True)

In [71]:
var_list = list(dfp)

In [72]:
retain = [ele for ele in var_list if ele not in unwanted]

In [73]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dimensions = ['student_id','disecode','student_name','class','section','roll_no'
                                  ,'unique_id','gender','social_category','medium_id'
                                  ,'Language_present_absent'
                                  ,'Maths_present_absent'
                                  ,'SS_present_absent'
                                  ,'Science_present_absent'
                                ]

In [74]:
dfp_unpivot = pd.melt(dfp,id_vars = dimensions
                        ,value_vars = retain)

In [75]:
dfp = pd.DataFrame()

In [76]:
#check
dfp_unpivot.shape

(31284108, 16)

In [77]:
dfp_unpivot[['Subject','Question']] = dfp_unpivot['variable'].str.split('Q',1,expand=True)

In [78]:
#Language mapped as Telugu for general files; English and Hindi will be separate scored files altogether
dfp_unpivot.loc[dfp_unpivot["Subject"]=="L","Subject"]="Telugu"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="M","Subject"]="Maths"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="E","Subject"]="EVS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="O","Subject"]="SS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="S","Subject"]="Science"

In [79]:
dfp_unpivot = dfp_unpivot.drop(['section','roll_no','student_id','variable'],axis=1)

In [80]:
dfp_unpivot.fillna('',inplace = True)

In [81]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
indexELM = dfp_unpivot[(((dfp_unpivot['Language_present_absent']=='A')|(dfp_unpivot['Language_present_absent']==''))&(dfp_unpivot['Subject']=='Telugu'))|
                       (((dfp_unpivot['Maths_present_absent']=='A')|(dfp_unpivot['Maths_present_absent']==''))&(dfp_unpivot['Subject']=='Maths'))|
                       (((dfp_unpivot['SS_present_absent']=='A')|(dfp_unpivot['SS_present_absent']==''))&(dfp_unpivot['Subject']=='SS'))|
                       (((dfp_unpivot['Science_present_absent']=='A')|(dfp_unpivot['Science_present_absent']==''))&(dfp_unpivot['Subject']=='Science'))
                      ].index

dfp_unpivot.drop(indexELM, inplace=True)

In [82]:
#check
dfp_unpivot.shape

(30746653, 14)

In [83]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dfp_unpivot = dfp_unpivot.drop(['Language_present_absent','Maths_present_absent','SS_present_absent','Science_present_absent'],axis=1)

In [84]:
dfp_unpivot['Question'] = dfp_unpivot['Question'].astype(np.int64)

In [85]:
dfp_unpivot = dfp_unpivot.rename(columns = {'unique_id':'Child_Id','disecode':'Udise','student_name':'Child_Name','class':'Grade','gender':'Gender','social_category':'Social_Category','medium_id':'Medium_Id','value':'Raw_Score','Subject':'Subject','Question':'Question_no'})

In [86]:
dfp_unpivot.to_pickle('./CBAScoredFiles/g8_scores.pkl')

<h1 style="color:blue;">
    Total Scores
</h1>

In [87]:
dfp = pd.read_csv('./CBAScoredFiles/g8.csv')

In [88]:
#check
dfp.shape

(411633, 190)

In [89]:
unwanted = ('student_id','disecode','student_name','class','section','roll_no','unique_id','gender','social_category','medium_id'
            ,'Language_paperset','Language_jumble_set','Language_present_absent','LanguageTotal','LanguagePercentage_old','LanguagePercentage','Language_cheating_drop'
            ,'Maths_paperset','Maths_jumble_set','Maths_present_absent','MathsTotal','MathsPercentage_old','MathsPercentage','Maths_cheating_drop'
            ,'EVS_paperset','EVS_jumble_set','EVS_present_absent','EVSTotal','EVSPercentage_old','EVSPercentage','EVS_cheating_drop'
            ,'Science_paperset','Science_jumble_set','Science_present_absent','ScienceTotal','SciencePercentage_old','SciencePercentage','Science_cheating_drop'
            ,'SS_paperset','SS_jumble_set','SS_present_absent','SSTotal','SSPercentage_old','SSPercentage','SS_cheating_drop'
           )

In [90]:
var_list = list(dfp)

In [91]:
#EQ = EVS, MQ = Maths, LQ = Language (Telugu), SQ = Science, OQ = Social Science
prefixes = ('EQ','MQ','LQ','SQ','OQ')
dropped = [x for x in var_list if x.startswith(prefixes)]

In [92]:
dfp.drop(dropped, axis = 1 , inplace = True)

In [93]:
var_list = list(dfp)

In [94]:
retain = [ele for ele in var_list if ele not in unwanted]

In [95]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dimensions = ['student_id','disecode','student_name','class','section','roll_no'
                                  ,'unique_id','gender','social_category','medium_id'
                                  ,'Language_present_absent'
                                  ,'Maths_present_absent'
                                  ,'SS_present_absent'
                                  ,'Science_present_absent'
                                ]

In [96]:
dfp_unpivot = pd.melt(dfp,id_vars = dimensions
                        ,value_vars = retain)

In [97]:
dfp = pd.DataFrame()

In [98]:
dfp_unpivot[['S','Q']] = dfp_unpivot['variable'].str.split('_',1,expand=True)

In [99]:
dfp_unpivot[['Subject','Question']] = dfp_unpivot['Q'].str.split('Q',1,expand=True)

In [100]:
#Language mapped as Telugu for general files; English and Hindi will be separate scored files altogether
dfp_unpivot.loc[dfp_unpivot["Subject"]=="L","Subject"]="Telugu"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="M","Subject"]="Maths"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="E","Subject"]="EVS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="O","Subject"]="SS"
dfp_unpivot.loc[dfp_unpivot["Subject"]=="S","Subject"]="Science"

In [101]:
dfp_unpivot = dfp_unpivot.drop(['section','roll_no','student_id','variable','S','Q'],axis=1)

In [102]:
dfp_unpivot.fillna('',inplace = True)

In [103]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
indexELM = dfp_unpivot[(((dfp_unpivot['Language_present_absent']=='A')|(dfp_unpivot['Language_present_absent']==''))&(dfp_unpivot['Subject']=='Telugu'))|
                       (((dfp_unpivot['Maths_present_absent']=='A')|(dfp_unpivot['Maths_present_absent']==''))&(dfp_unpivot['Subject']=='Maths'))|
                       (((dfp_unpivot['SS_present_absent']=='A')|(dfp_unpivot['SS_present_absent']==''))&(dfp_unpivot['Subject']=='SS'))|
                       (((dfp_unpivot['Science_present_absent']=='A')|(dfp_unpivot['Science_present_absent']==''))&(dfp_unpivot['Subject']=='Science'))
                      ].index

dfp_unpivot.drop(indexELM, inplace=True)

In [104]:
#check
dfp_unpivot.shape

(30746653, 14)

In [105]:
#Define dimensions based on grade - present absent flag will change based on the subjects relevant to that grade
#G2 - Maths, Language; G3,4,5 - EVS, Maths, Language; G6,7,8 - Maths, Language, SS, Science
dfp_unpivot = dfp_unpivot.drop(['Language_present_absent','Maths_present_absent'],axis=1)

In [106]:
dfp_unpivot['Question'] = dfp_unpivot['Question'].astype(np.int64)

In [107]:
dfp_unpivot = dfp_unpivot.rename(columns = {'unique_id':'Child_Id','disecode':'Udise'\
                                            ,'student_name':'Child_Name','class':'Grade'\
                                            ,'gender':'Gender','social_category':'Social_Category'\
                                            ,'medium_id':'Medium_Id','value':'Total_Score'\
                                            ,'Subject':'Subject','Question':'Question_no'})

In [108]:
dfp_unpivot.to_pickle('./CBAScoredFiles/g8_raw_scores.pkl')

In [109]:
dfp_unpivot = pd.DataFrame()

<h1 style="color:blue;">
    Merge files
</h1>

In [110]:
Scores = pd.read_pickle('./CBAScoredFiles/g8_scores.pkl')

In [111]:
Raw = pd.read_pickle('./CBAScoredFiles/g8_raw_scores.pkl')

In [112]:
df = pd.merge(Scores, Raw, on=['Child_Id','Udise','Child_Name','Grade','Gender','Social_Category'\
                               ,'Medium_Id','Subject','Question_no'], suffixes=("", ""))


<h1 style="color:blue;">
    School mapping
</h1>

In [113]:
school_mapping = pd.read_excel('./MappingFiles/AP_SchoolMaster_DistrictMandal_Mapping.xlsx')

In [114]:
df = pd.merge(df,school_mapping,how='left',left_on='Udise',right_on='SCHOOLCODE')

In [115]:
df = df.assign(State='Andhra Pradesh',Topic='',Subtopic='',Skill='',MUA='')

In [116]:
df = df.rename(columns={'SCHOOLNAME':'School','schooltype':'School_Type','Udise':'Udise'\
                                       ,'Social_Category':'Social_Category','District ':'District'})

In [117]:
df = df.drop(['SCHOOLCODE','Medium_Id'],axis=1)

In [118]:
df = df[[
'State',
'District',
'Mandal',
'School',
'Udise',
'School_Type',
'Grade',
'Child_Name',
'Child_Id',
'Gender',
'Social_Category',
'Subject',
'Question_no',
'Total_Score',
'Raw_Score',
'Topic',
'Subtopic',
'Skill',
'MUA']]

In [119]:
df.to_pickle('./CBALongPickleFiles/G8.pkl')

In [120]:
#df.to_csv('./CBALongFiles/CBA_G8_LongFile.csv')

In [121]:
df = pd.DataFrame()